In [ ]:
# Library
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import string
import time
import re
import requests
from bs4 import BeautifulSoup
from twitterscraper import query_tweets
import datetime as dt
import numpy as np

#******************************************************************************
# Crawling historical prices

df1 = pd.read_csv('stockname.csv',usecols=[1])
datanames = df1['name'].tolist()

sl=[];cl=[];ol=[];hl=[];ll=[];dl=[];vl=[];stocknames2=[]  

#set timeframe to crawl e.g. 3 months
startdate=str(1546343431) #date = Tuesday, January 1, 2019 7:50:31 PM
enddate=str(1554205831) #date = Tuesday, April 2, 2019 7:50:31 PM 

for name in datanames:
    url = 'https://charts.thestar.com.my/datafeed-udf/history?symbol='+name+'&resolution=D&from='+startdate+'&to='+enddate
    r = requests.get(url).json() 
    if r["s"] == "ok":
        stocknames2.append(name)
        for t in r["t"]:
            day=time.strftime("%Y-%m-%d",time.localtime(int(t)))
            dl.append(day)
            sl.append(name)
        for o in r["o"]:ol.append(o) #open price
        for c in r["c"]:cl.append(c) #closing price
        for h in r["h"]:hl.append(h) #high price
        for l in r["l"]:ll.append(l) #low price
        for v in r["v"]:vl.append(v) #volume
    print("Done for "+ name)
    #break       
    
df = pd.DataFrame({'name':sl,'day':dl,'close':cl,'open':ol,'high':hl,'low':ll,'volume':vl})
df.to_csv('price_df.csv')


#*****************************************************************************
# Crawl financial information)

browser = webdriver.Chorome(executable_path=r'C:\Users\khnam\Anaconda3\geckodriver.exe')
browser.implicitly_wait(40)
Financialurl = 'https://klse.i3investor.com/financial/quarter/latest.jsp'
browser.get(Financialurl)
WebElementexpanded = browser.find_element_by_xpath("//*[@id='ui-accordion-financialResultTableColumnsDiv-header-0']/span")
WebElementexpanded.click()


allLinks = browser.find_elements_by_xpath('//input[@type="checkbox"]')
for link in allLinks:
    if link.is_selected():
        print('Checkbox already selected');
    else:
        link.click();
        print('Checkbox selected'); 

df = pd.DataFrame(columns=['no', 'stockname', 'annDate', 'fy', 
                           'quarter', 'h', 'price', 'ch', 'percentage', 
                           'revenue', 'pbt', 'np', 'NptoSh', 'dividend', 
                           'networth', 'divpayout', 'npmargin', 'roe', 
                           'rps', 'eps', 'dps', 'naps', 'QoY', 'YoY'])

elm = browser.find_element_by_class_name('next')
tbl = browser.find_element_by_xpath('//*[@id="tablebody"]')
while True:
    element = WebDriverWait(browser, 100).until(lambda x: x.find_element_by_id('tablebody'))
    for row in tbl.find_elements_by_tag_name('tr'):
        data = row.find_elements_by_tag_name('td')
        file_row = []
        for datum in data:
            datum_text = datum.text
            file_row.append(datum_text)
        print(file_row)
        s = pd.Series(file_row, index = df.columns)
        df = df.append(s,ignore_index=True)
    elm = browser.find_element_by_class_name('next')
    if 'ui-state-disabled' in elm.get_attribute('class'):
        break;
    elm.click()      
        
df = df.drop(columns='no')
df.to_csv('financial_df.csv',index=False)



